In [1]:
from facebookads.adobjects.customaudience import CustomAudience
from facebookads.adobjects.adaccount import AdAccount
from facebookads.api import FacebookAdsApi
import json
import pandas as pd
import os
import datetime
import numpy as np

BigLotsAccount_id="act_271491453638620"
BigLotsDcart10_id="act_2449979771956630"

list_account_id=[BigLotsAccount_id,BigLotsDcart10_id]
# jian_APP_ID="2537704939796694"# jian's fb marketing app id
jian_token=json.load(open("/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/FB_token/lasest_FB_token_jian.json","r"))['latest_FB_token']

In [2]:
# create new adueince and add the new users to the audiecne, 1 step
# Please note the change of the CustomAudience.Schema (row 41 based on the user list format)
def creatfbaudience(AudienceName,
                    AudienceDescription,
                    df,
                    ACCESS_TOKEN,
                    accountid,
                    df_use_col_name,
                    fb_schema=CustomAudience.Schema.email_hash):
    FacebookAdsApi.init(access_token=ACCESS_TOKEN, api_version='v8.0')
    
    '''
    AudienceName: str, the name shown on FB
    AudienceDescription: str, the description shown on FB
    df: pd.Dataframe, input with the identifier col
    ACCESS_TOKEN: str, access to FB marketing api
    accountid: str, account to uplaod
    df_use_col_name: str, col name in the df
    fb_schema: available schema attribution in the CustomAudience.Schema class. Default for us CustomAudience.Schema.email_hash  
    https://github.com/facebook/facebook-python-business-sdk/blob/ac390147132101763a99a917bb59bdc8a374a7ce/facebook_business/adobjects/helpers/customaudiencemixin.py
    '''
    
    myAccount = AdAccount(accountid)
    fields = [
    ]
    params = {
      'name': AudienceName,
      'subtype': 'CUSTOM',
      'description': AudienceDescription,
      'customer_file_source': 'PARTNER_PROVIDED_ONLY',
    }
    fbname = myAccount.create_custom_audience(
      fields=fields,
      params=params,
    )['id']
    print("Audience ID:",fbname)
    audience = CustomAudience(fbname)
    users = list(df[df_use_col_name])
    listlen = len(users)
    chunck_size=50000
    count_uploaded=0
    for i in range(int(np.ceil(listlen/chunck_size))):
        starti = i*50000
        if (i+1)*50000<listlen:
            endi = (i+1)*50000
        else:
            endi = listlen
        list0 = users[starti:endi]
        
        count_uploaded+=len(list0)
        audience.add_users(fb_schema, list0)
        #print len(list0)   
    print(datetime.datetime.now(),AudienceName,"uploaded",count_uploaded)


In [3]:
import glob
folder_client_segs="./"
list_files_to_upload_10_off_20=glob.glob(folder_client_segs+"*.csv")
# list_files_to_upload_PYD=[x for x in list_files_to_upload_PYD if "_C.csv" not in x]
list_files_to_upload_10_off_20.sort()
list_files_to_upload_10_off_20

['./10-12 Month About to Lapse MS A_20201102075436.csv',
 './10-12 Month About to Lapse MS B_20201102075837.csv',
 './13-18 Month Lapsed MS A_20201102073523.csv',
 './13-18 Month Lapsed MS B_20201102073920.csv',
 './Online Signup MS A_20201102074635.csv',
 './Online Signup MS B_20201102074942.csv']

In [4]:
pd.read_csv(list_files_to_upload_10_off_20[0],nrows=10,dtype=str)

,customersummary_mastercustomerid,customersummary_emailhash,customersummary_c_primaryscnhash,execution_id
0,KFK_0_58796144,25abf112b00caffcbd9aef32d66bd67a80b853ce039dcb...,068b3d7bc1f50214e5f0a6e05b8565359687334cddbcf2...,1604346769164
1,KFK_0_10898437,976d2bcdbab584263442ea5cc4fb28c0d318f13a4be6b4...,af002365ba37f81669d9db9b61b2f6c5a2f6e3bf4f2bbc...,1604346769164
2,FTP_CSV_6_58611036,593b09df84abf0e8dfd79297c6eeaa2a32f0238f592c68...,4793add6a2f4a325649c31415ae21450034d7ee53aace5...,1604346769164
3,FTP_CSV_6_55259218,014986a39ec7c0f4e1f8f504da5e64c6e19c2a9d6fb72b...,4dd3079297406d44ebd25a086ec6bf6f93b5282ed52044...,1604346769164
4,FTP_CSV_6_58766382,399befdd05497a1d304d741015b414319639456896cb65...,947a108ae443c974fc467a88392c38611730675f8de32c...,1604346769164
5,KFK_0_57741185,e8c6d8b39784e609fc353ef905dbd3ab75db32ccb24fd3...,8cb458cf58f02962010a35461a0824981340678440ecd3...,1604346769164
6,FTP_CSV_6_9319639,6af87f8652594eb4db5c47040bd18c38c551a36fde6084...,f5ede8f4e55020b4d884e9900b4df34c61f2344139d51a...,1604346769164
7,FTP_CSV_6_50135921,9197d472adf6dcb0e46d1930566fd6e44409508f1e476c...,85a203ae1ba6a19fe9575e82f8a788830073084ab677a6...,1604346769164
8,FTP_CSV_6_29349017,120b5fd6b61d9d9c83fe5cae59773eb686c95cf138bccb...,6f3d1f7a5ee6c8efe0b6536d6eeb2043fe45f680b733a4...,1604346769164
9,FTP_CSV_6_39078568,104b605e2f6a4ae6b7690ae515184e594145733f930bf1...,2563a77fbd0f13850f97c514a424ab20a0c84b280883df...,1604346769164


In [9]:
col_identifer='customersummary_emailhash'
i=0
df_summary=pd.DataFrame()
for file in list_files_to_upload_10_off_20:
    for fb_account_id in list_account_id:
        i+=1
        audience_name = os.path.basename(file).split(".")[0]
        description=os.path.basename(file).split(".")[0]+"_uploaded_by_Juba_"+str(datetime.datetime.now().date())
        df = pd.read_csv(file,dtype=str)
        row_count=df.shape[0]
        df=df[df[col_identifer].apply(len)==64]
        email_unique_count=df[col_identifer].nunique()
        df_summary_i=pd.DataFrame({'FB_account':fb_account_id,"audience_name":audience_name,"row_count":row_count,"unique_id":email_unique_count},index=[i])
        df_summary=df_summary.append(df_summary_i)
        print(audience_name, email_unique_count)
        creatfbaudience(AudienceName="aud_"+audience_name,
                        AudienceDescription=description+", file received from client on 20200915, 107 agile",
                        df=df,
                        df_use_col_name=col_identifer,
                        ACCESS_TOKEN=jian_token,
                        accountid=fb_account_id)

10-12 Month About to Lapse MS A_20201102075436 2405574
Audience ID: 23845836194910265
2020-11-03 01:16:39.724117 aud_10-12 Month About to Lapse MS A_20201102075436 uploaded 2405616
10-12 Month About to Lapse MS A_20201102075436 2405574
Audience ID: 23846192280570632
2020-11-03 01:18:51.911223 aud_10-12 Month About to Lapse MS A_20201102075436 uploaded 2405616
10-12 Month About to Lapse MS B_20201102075837 223905
Audience ID: 23845836211360265
2020-11-03 01:19:06.945864 aud_10-12 Month About to Lapse MS B_20201102075837 uploaded 223905
10-12 Month About to Lapse MS B_20201102075837 223905
Audience ID: 23846192285420632
2020-11-03 01:19:21.832596 aud_10-12 Month About to Lapse MS B_20201102075837 uploaded 223905
13-18 Month Lapsed MS A_20201102073523 3541018
Audience ID: 23845836216340265
2020-11-03 01:23:04.466994 aud_13-18 Month Lapsed MS A_20201102073523 uploaded 3541111
13-18 Month Lapsed MS A_20201102073523 3541018
Audience ID: 23846192299680632
2020-11-03 01:26:19.896049 aud_13-18 

In [10]:
df_summary

,FB_account,audience_name,row_count,unique_id
1,act_271491453638620,10-12 Month About to Lapse MS A_20201102075436,2405635,2405574
2,act_2449979771956630,10-12 Month About to Lapse MS A_20201102075436,2405635,2405574
3,act_271491453638620,10-12 Month About to Lapse MS B_20201102075837,223906,223905
4,act_2449979771956630,10-12 Month About to Lapse MS B_20201102075837,223906,223905
5,act_271491453638620,13-18 Month Lapsed MS A_20201102073523,3541148,3541018
6,act_2449979771956630,13-18 Month Lapsed MS A_20201102073523,3541148,3541018
7,act_271491453638620,13-18 Month Lapsed MS B_20201102073920,341226,341223
8,act_2449979771956630,13-18 Month Lapsed MS B_20201102073920,341226,341223
9,act_271491453638620,Online Signup MS A_20201102074635,3600659,3600594
10,act_2449979771956630,Online Signup MS A_20201102074635,3600659,3600594


In [11]:
df_summary[['audience_name','row_count','unique_id']].drop_duplicates()

,audience_name,row_count,unique_id
1,10-12 Month About to Lapse MS A_20201102075436,2405635,2405574
3,10-12 Month About to Lapse MS B_20201102075837,223906,223905
5,13-18 Month Lapsed MS A_20201102073523,3541148,3541018
7,13-18 Month Lapsed MS B_20201102073920,341226,341223
9,Online Signup MS A_20201102074635,3600659,3600594
11,Online Signup MS B_20201102074942,359670,359665
